# Generalized Fisher's Linear Discriminant - Exploring Numerical Issues

Some weird numerical issues seem to arise  (at least, weird according to the original fisherface paper)

## Set up

In [1]:
import numpy as np
import sklearn.pipeline
import matplotlib.pyplot as plt

In [2]:
# load data
import sklearn.datasets
from sklearn.model_selection import train_test_split


lfw_people = sklearn.datasets.fetch_lfw_people(min_faces_per_person=70, resize=0.4)

n_samples, h, w = lfw_people.images.shape
target_names = lfw_people.target_names
n_classes = target_names.shape[0]

X = lfw_people.data
n_features = X.shape[1]

y = lfw_people.target


print("Total dataset size:")
print("n_samples: ",  n_samples)
print("n_features: ", n_features)
print("n_classes: ", n_classes)



# split into a training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# first, scale the data
scaler = sklearn.preprocessing.StandardScaler().fit(X_train)
X_train_trfm = scaler.transform(X_train)
X_test_trfm = scaler.transform(X_test)

n_samples_train = len(X_train)
print('# of training samples: ', n_samples_train)

Total dataset size:
n_samples:  1288
n_features:  1850
n_classes:  7
# of training samples:  966


## Rank of $S_w$ and $S_b$

Note that the number of training samples (966) is less than the number of features/dimensionality of images (1850). This is the same as the set up in the fisherface paper.

The fisherace paper says: "In the face recognition problem, one is confronted with the difficulty that the within-class scatter matrix $S_W \in \mathbb{R}^{n \times n}$ is always singular. This stems from the fact that the rank of $S_W$ is at most $N - c$, and, in general, the number of images in the learning set $N$ is much smaller than the number of pixels in each image $n$."

**Q:** How do they arrive at the claim above? Why is it true?

In [3]:
from generalized_fisher_ld import GeneralizedFisherLD
import sklearn.pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA

fld_clf = sklearn.pipeline.Pipeline(steps=[#('pca', PCA(n_components = n_samples_train - (n_classes - 1))),
                                           ('fld', GeneralizedFisherLD(n_components=n_classes - 1, alpha=0, shrinkage='auto')),
                                           ('rf', RandomForestClassifier())])
fld_clf.fit(X_train_trfm, y_train)

Pipeline(steps=[('fld', GeneralizedFisherLD(n_components=6, shrinkage='auto')),
                ('rf', RandomForestClassifier())])

In [7]:
S_w, S_b = fld_clf[0].S_w, fld_clf[0].S_b
print('det(S_w) = ', np.linalg.det(S_w))
print('det(S_b) = ', np.linalg.det(S_b))

print()
print('rank(S_w) = ', np.linalg.matrix_rank(S_w))
print('rank(S_b) = ', np.linalg.matrix_rank(S_b))

det(S_w) =  0.0
det(S_b) =  -0.0

rank(S_w) =  1850
rank(S_b) =  1850


Both $S_w$ and $S_b$ have "0" determinant (as far numpy's precision), yet they have full rank...

Furthermore, all 1850 eigenvalues are non-zero.

In [11]:
import scipy
eig_values, eig_vecs = scipy.linalg.eigh(S_b, S_w)

In [22]:
eig_values

array([-0.84546796, -0.84354609, -0.84307492, ...,  3.39043189,
        4.71632741,  5.60518439])

In [17]:
np.sum(eig_values != 0)

1850

In [21]:
# V^T S_w V = Identity...
eig_vecs.T @ S_w @ eig_vecs

array([[ 1.00000000e+00, -7.60538008e-15,  3.07681669e-15, ...,
        -2.69749501e-16, -3.66135074e-16,  1.62820231e-15],
       [-4.40412409e-15,  1.00000000e+00,  1.01318693e-15, ...,
        -6.06251974e-16,  7.61435186e-16, -5.13369729e-16],
       [ 2.77103779e-15,  5.73434529e-16,  1.00000000e+00, ...,
        -5.30066442e-16, -1.16226473e-16, -3.65050871e-16],
       ...,
       [ 2.88560408e-16, -1.52568930e-15, -2.43566018e-16, ...,
         1.00000000e+00, -1.69309011e-15, -3.26128013e-16],
       [-1.04170145e-15,  1.43548368e-15, -1.84227633e-15, ...,
        -1.42941214e-15,  1.00000000e+00, -9.64506253e-16],
       [ 1.40686074e-15, -1.97758476e-15, -1.74860126e-15, ...,
        -1.15879528e-15, -1.65839564e-15,  1.00000000e+00]])

**Q:** How to reconcile/interpret discrepency in maximum possible components? It seems we can use up to $n$ components just as in PCA/eigenface...

Finally, for the generalized eigenvector problem given in the original fisherface paper, $S_b w_i = \lambda_i S_w w_i$, as above, we were able to directly compute the eigenvectors. But for the reverse problem, $S_w w_i = \lambda_i S_b w_i$, we do get an regarding the rank of the matrices... this is what we would have expected from the original problem.

In [23]:
eig_values, eig_vecs = scipy.linalg.eigh(S_w, S_b)

LinAlgError: The leading minor of order 2 of B is not positive definite. The factorization of B could not be completed and no eigenvalues or eigenvectors were computed.

## Try for sklearn's implementation of LDA as a sanity check...

In [27]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import sklearn.pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA

fld_clf = sklearn.pipeline.Pipeline(steps=[('pca', PCA(n_components = n_samples_train - (n_classes - 1))),
                                           ('fld', LinearDiscriminantAnalysis(n_components=n_classes - 1, solver='eigen')),# shrinkage='auto')),
                                           ('rf', RandomForestClassifier())])
fld_clf.fit(X_train_trfm, y_train)

Pipeline(steps=[('pca', PCA(n_components=960)),
                ('fld',
                 LinearDiscriminantAnalysis(n_components=6, solver='eigen')),
                ('rf', RandomForestClassifier())])

In [28]:
fld_clf.score(X_test_trfm, y_test)

0.43788819875776397

LinAlg Error does not occur when `shrinkage='auto'` (which is what I used in my implementation), but does occur otherwise and is fixed by performing PCA first! This is as expected...

## Exploration for `shrinkage != 'auto'`
more what's expected

In [30]:
from generalized_fisher_ld import GeneralizedFisherLD
import sklearn.pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA

fld_clf = sklearn.pipeline.Pipeline(steps=[('pca', PCA(n_components = n_samples_train - (n_classes - 1))),
                                           ('fld', GeneralizedFisherLD(n_components=n_classes - 1, alpha=0, shrinkage=None)),
                                           ('rf', RandomForestClassifier())])
fld_clf.fit(X_train_trfm, y_train)

Pipeline(steps=[('pca', PCA(n_components=960)),
                ('fld', GeneralizedFisherLD(n_components=6)),
                ('rf', RandomForestClassifier())])

In [31]:
fisherfaces = (fld_clf[1].transformation_matrix_ @ fld_clf[0].components_).T

In [37]:
np.linalg.det(fisherfaces @ fld_clf[1].S_w @ fisherfaces.T)

0.0

In [38]:
np.linalg.det(fisherfaces @ fld_clf[1].S_b @ fisherfaces.T)

4.2756468489018215e+209

In [39]:
np.linalg.matrix_rank(fld_clf[1].S_w)

959

In [40]:
np.linalg.matrix_rank(fld_clf[1].S_b)

6

TODO:
- test for $\alpha = \beta \neq 0$
- test # of non-zero eigenvalues (i.e.: max number of components)
- etc.

In [43]:
from generalized_fisher_ld import GeneralizedFisherLD
import sklearn.pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA

fld_clf = sklearn.pipeline.Pipeline(steps=[#('pca', PCA(n_components = n_samples_train - (n_classes - 1))),
                                           ('fld', GeneralizedFisherLD(n_components=n_classes - 1, alpha=0.5, beta=0.5, shrinkage=None)),
                                           ('rf', RandomForestClassifier())])
fld_clf.fit(X_train_trfm, y_train)

Pipeline(steps=[('fld',
                 GeneralizedFisherLD(alpha=0.5, beta=0.5, n_components=6)),
                ('rf', RandomForestClassifier())])

In [44]:
fisherfaces = fld_clf[0].transformation_matrix_

In [53]:
np.linalg.det(fisherfaces @ fld_clf[0].B_matrix_ @ fisherfaces.T)

1.000000000000056

In [54]:
np.linalg.det(fisherfaces @ fld_clf[0].A_matrix_ @ fisherfaces.T)

0.0

In [56]:
print('rank(S_b) = ', np.linalg.matrix_rank(fld_clf[0].S_b))
print('rank(S_w) = ', np.linalg.matrix_rank(fld_clf[0].S_w))

print()
print('rank(A) = ', np.linalg.matrix_rank(fld_clf[0].A_matrix_))
print('rank(B) = ', np.linalg.matrix_rank(fld_clf[0].B_matrix_))

rank(S_b) =  6
rank(S_w) =  959

rank(A) =  965
rank(B) =  1850


In [57]:
eig_vals, eig_vecs = scipy.linalg.eigh(fld_clf[0].A_matrix_, fld_clf[0].B_matrix_)

In [72]:
print(np.sum(eig_vals[::-1] > 0.1)) # note that we get more components!!

362


**Q:** How many components do we get? $\min(rank(A), rank(B))$?